# Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

# This is needed to display the images.
%matplotlib inline

## Object detection 시각화 모듈 import 

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model 준비

In [5]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## 모델 loading

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection 함수 정의

In [9]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = 'test_images'
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
import glob

TEST_IMAGE_PATHS=glob.glob('./images/*.jpg')
#os.listdir('/var/www/html/img/quiz_num')
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [10]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [11]:
import json
import random
import collections
import pyjosa

In [12]:
import pandas
xls = pandas.ExcelFile('/home/ktai19/Downloads/engtokor.xlsx')
df = xls.parse(xls.sheet_names[0])
things=df.korean.values.tolist()
#print df.to_dict()

In [13]:
def kind(class_name):
    if df[df.korean==class_name].kind.values[0]==1 :return "명"
    elif df[df.korean==class_name].kind.values[0]==2 :return "마리"
    elif df[df.korean==class_name].kind.values[0]==3 :return "개"

In [14]:
# 한 object가 7번 이상 등장하면 고급으로 분류
def det_level_by_n(classname):
    if cnt[classname]>=7:return 3
    elif cnt[classname]<7 and cnt[classname]>=4:return 2
    else: return 1

#한 사진에서 detect된 object가 3종류 이상이면 고급
def det_level_by_nkind(objects):
    if len(objects)>=3:return 3
    elif len(objects)==2:return 2
    else: return 1


In [15]:
import pickle
def save_obj(obj, name ):
    with open('./detection_result/'+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./detection_result/'+name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
with open('./data.json','r') as file:
    datastore=json.load(file)
previous_files=list(set([quest['FileName'] for quest in datastore]))

# 사진 분석하고 분석결과 저장하기

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    FileName=os.path.basename(image_path)
    print(FileName)
    if FileName in previous_files:
        continue
    #image_path="/var/www/html/img/quiz_num/22.jpg"  
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
#    output_dict['FileName']=os.path.basename(image_path)
    classes=output_dict.get('detection_classes')
    boxes=output_dict.get('detection_boxes')
    scores=output_dict.get('detection_scores')
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
    #ymin , xmin, ymax, xmax
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    
    save_obj(output_dict,FileName)

    

12.jpg
11.jpg
17.jpg


# detection result를 이용해 문제만들기

In [ ]:
data=[]
for file in os.listdir('./detection_result/'):
    FileName=file.strip('.pkl')
    if FileName in previous_files:
        continue
    result=load_obj(FileName)
    classes=result.get('detection_classes')
    boxes=result.get('detection_boxes')
    scores=result.get('detection_scores')
    objects=[]
    locations=[]
    for i in range(min(20, boxes.shape[0])):
        if scores[i]>.5:
            class_name = category_index[classes[i]]['name']
            class_name=df[df.english==class_name].korean.values[0]
            objects.append(class_name)
            locations.append((boxes[i][1]+boxes[i][3],boxes[i][3]))

    cnt=collections.Counter(objects)
    class_name=cnt.most_common()[0][0]
    print(FileName)

    unique_obj=list(set(objects))
    
    #1. 사진 속 최빈 물체 몇개인지에 관한 문제
    def q1():
        ans= str(cnt[class_name])+kind(class_name) #5명
        bias=random.sample([-2,-1,1,2],2)
        cand1= str(cnt[class_name]+bias[0])+kind(class_name) # 3명~4명
        cand2= str(cnt[class_name]+bias[1])+kind(class_name) #6~7명
        answers=[ans,cand1,cand2]
        random.shuffle(answers)
        data.append({
            "Answer":str(answers.index(ans)+1)+"번,"+ans,
            "Question":pyjosa.replace_josa("사진 속 {}(은)는 몇 {}인가요?".format(class_name,kind(class_name))),
            "FileName":FileName,
            "Level":det_level_by_n(class_name),
            "ans1":answers[0],
            "ans2":answers[1],
            "ans3":answers[2]
        })
    q1()

    #2. 사진 속 가장 많은 것은 무엇?
    def q2():
        if len(unique_obj)>=3:
            ans= class_name # 우산
            candidates=unique_obj[:]
            candidates.remove(ans)
            cand1,cand2=random.sample(candidates,2)
            answers=[ans,cand1,cand2]
            random.shuffle(answers)
            data.append({
                "Answer":str(answers.index(ans)+1)+"번,"+ans,
                "Question":"사진 속에서 가장 많이 등장한 것은 어떤 것인가요?",
                "FileName":FileName,
                "Level":1,
                "ans1":answers[0],
                "ans2":answers[1],
                "ans3":answers[2]
            })
    q2()
            
    #3. 사진 속 " "(동물,물체)는 무엇인가요?
    def q3():
        if len(unique_obj)==1:
            ans=class_name
            cand1,cand2=random.sample(things,2)
            answers=[ans,cand1,cand2]
            random.shuffle(answers)
            data.append({
                "Answer":str(answers.index(ans)+1)+"번,"+ans,
                "Question":"사진에 보이는 것은 무엇인가요?",
                "FileName":FileName,
                "Level":1,
                "ans1":answers[0],
                "ans2":answers[1],
                "ans3":answers[2]
            })
    q3()

    #4. 다음 중 사진에 없었던 것은 무엇인가요?
    def q4():
        if len(unique_obj)>=2:
            ans=random.choice([el for el in things if el not in unique_obj])# 우산
            cand1,cand2=random.sample(unique_obj,2)
            answers=[ans,cand1,cand2]
            random.shuffle(answers)
            data.append({
                "Answer":str(answers.index(ans)+1)+"번,"+ans,
                "Question":"다음 중 사진에 없었던 것은 무엇인가요?",
                "FileName":FileName,
                "Level":det_level_by_nkind(objects),
                "ans1":answers[0],
                "ans2":answers[1],
                "ans3":answers[2]
            })
    q4()
    #5. 위치 물어보기
    def q5():
        if len(unique_obj)>=3:
            
            def get_max_loc(locations): #중점과 오른쪽 점이 모두 다른 object보다 오른쪽에 있는 물체가 있을때만 문제를 출제하자
                #if one of element has max sum and max right: return that el's idx'
                loc_sum=[el[0] for el in locations]
                l1=loc_sum.index(max(loc_sum))

                loc_right=[el[1] for el in locations]
                l2=loc_right.index(max(loc_right))
                if l1==l2:
                    return l1
                else:
                    return None

            maxidx=get_max_loc(locations)
            if maxidx:
                ans=objects[maxidx]#random.choice([el for el in things if el not in objects])# 우산
                candidates=objects[:maxidx]+objects[maxidx+1:]
                cand1,cand2=random.sample(set(list(candidates)),2)
                answers=[ans,cand1,cand2]
                random.shuffle(answers)
                data.append({
                    "Answer":str(answers.index(ans)+1)+"번,"+ans,
                    "Question":"다음 중 사진에서 가장 오른쪽에 있던 물체는 무엇인가요?",
                    "FileName":FileName,
                    "Level":det_level_by_nkind(objects),
                    "ans1":answers[0],
                    "ans2":answers[1],
                    "ans3":answers[2]
                })
    q5()

In [ ]:
def append_json(data):
    with open('./data.json', 'r') as file:  
        prev=json.load(file)
        if data not in prev:
            prev.extend(data)
        else:
            print("skip")
    with open('./data.json', 'w') as outfile:  
        json.dump(prev, outfile)

In [ ]:
if os.path.exists('./data.json'):
    append_json(data)
else:
    with open('./data.json', 'w') as outfile:  
        json.dump(data, outfile)

# 다중사진 문제 만들기

In [ ]:
import itertools

In [ ]:
data=[]

comb=(itertools.combinations(TEST_IMAGE_PATHS,3))
comb=[list(map(os.path.basename,c)) for c in comb]

test=[','.join(el) for el in comb]

set_problems=[el for formated,el in zip(test,comb) if formated not in previous_files]

In [ ]:
set_problems

In [ ]:
# for file in os.listdir('./detection_result/'):
#     FileName=file.strip('.pkl')
for image_set in set_problems:
    cumulated_objs=[]
    for FileName in image_set:
        result=load_obj(FileName)
        classes=result.get('detection_classes')
        boxes=result.get('detection_boxes')
        scores=result.get('detection_scores')
        objects=[]
        locations=[]

        objects=[]
        for i in range(min(20, boxes.shape[0])):
            if scores[i]>.6:
                class_name = category_index[classes[i]]['name']
                class_name=df[df.english==class_name].korean.values[0]
                objects.append(class_name)
        
        cumulated_objs.extend(objects)
    fileset=','.join(map(os.path.basename,image_set))
    if fileset in previous_files:
        print(fileset," is already in problem set")
        continue
    #6. 보였던(2~3장) 사진들 중 없었던 것은?
    def q6():
        ans=random.choice([el for el in things if el not in cumulated_objs])# object에 계속 append 하자.
        try:
            cand1,cand2=random.sample(list(set(cumulated_objs)),2)
        except:
            return None
        answers=[ans,cand1,cand2]
        random.shuffle(answers)
        data.append({
            "Answer":str(answers.index(ans)+1)+"번,"+ans,
            "Question":"보였던 사진들 중 없었던 것은 무엇인가요?",
            "FileName":fileset,
            "Level":det_level_by_nkind(cumulated_objs),
            "ans1":answers[0],
            "ans2":answers[1],
            "ans3":answers[2]
        })
    q6()
    
    #7.보였던(2~3장) 사진들 중에서 무엇이 가장 많이 등장했나요?
    #class_name=random.choice(objects)
    def q7():
        cnt=collections.Counter(cumulated_objs)
        if cnt.most_common()[0][1]==cnt.most_common()[1][1]:
            print('no one maximum')
            return 0
        else:
            class_name=cnt.most_common()[0][0]
            ans= class_name # 우산
            #kind=df[df.korean==class_name].kind.values[0]
            if len(cnt.keys())>=3:
                cand1=cnt.most_common()[1][0]
                cand2=cnt.most_common()[2][0]
            else:
                return None
            answers=[ans,cand1,cand2]
            random.shuffle(answers)
            data.append({
                "Answer":str(answers.index(ans)+1)+"번,"+ans,
                "Question":"사진 속에서 가장 많이 등장한 것은 어떤 것인가요?",
                "FileName":fileset,
                "Level":1,
                "ans1":answers[0],
                "ans2":answers[1],
                "ans3":answers[2]
            })
    q7()

In [ ]:
append_json(data)

In [ ]:
data